In [3]:
# Import necessary modules from Biopython and other libraries
import Bio
import Bio.Blast
from Bio import Entrez, SeqIO  # Entrez for accessing NCBI databases, SeqIO for sequence input/output
from Bio.Blast import NCBIWWW  # NCBIWWW for running BLAST searches online
from Bio.Blast import NCBIXML  # NCBIXML for parsing BLAST XML results
from io import StringIO  # StringIO for handling string input/output

def get_protein_sequence(protein_id: str) -> str:
    """
    Retrieve protein sequence from NCBI using protein ID (accession number).
    Args:
        protein_id (str): NCBI protein ID.
    Returns:
        str: Protein sequence.
    """
    # Prompt the user to provide their email address (required by NCBI)
    Entrez.email = input("Provide your email address (required by NCBI): ")
    
    try:
        # Retrieve the protein sequence from the NCBI database using the provided protein_id
        handle = Entrez.efetch(
            db="protein",  # Specifies the database to search
            id=protein_id,  # Uses the input protein_id here
            rettype="fasta",  # Specifies the return type as FASTA
            retmode="text"  # Specifies the return mode as text
        )
        
        # Read the sequence data from the handle
        record = SeqIO.read(handle, "fasta")  # Parse the FASTA data into a SeqRecord
        handle.close()  # Close the handle to free resources
        
        return str(record.seq)  # Return the protein sequence as a string variable
    
    except Exception as e:
        # Print an error message if fetching the sequence fails
        print(f"Error fetching sequence: {e}")
        return None  # Return None if there was an error

def run_blast_search(protein_sequence: str) -> list:
    """
    Run BLASTp search for a given protein sequence (option for a protein sequence already retrienved 
    with the previous function) and return top 100 hits.
    Args:
        protein_sequence (str): Protein sequence in FASTA format.        
    Returns:
        Top 100 BLAST hits sorted by query coverage and a list of their accession numbers.
    """
    # Prompt the user to provide their email address (required by NCBI)
    Entrez.email = input("Provide your email address (required by NCBI): ")
    
    try:
        # Run BLAST search
        print("Running BLAST search (this may take several minutes)...")
        result_handle = NCBIWWW.qblast(
            program="blastp",  # Specifies the BLAST program to use
            database="nr",  # Specifies the database to search (non-redundant protein sequences)
            sequence=protein_sequence,  # Uses the provided protein sequence
            hitlist_size=100,  # Limits the number of hits returned to 100
            expect=10,  # Sets the E-value threshold for reporting hits
            matrix_name="BLOSUM62"  # Specifies the scoring matrix to use
        )
        
        # Parse results
        blast_records = NCBIXML.parse(result_handle)  # Parse the BLAST XML results
        blast_record = next(blast_records)  # Get the first BLAST record
        
        # Store results
        results = []  # Initialize a list to store the results
        accession_numbers = []  # New list for accessions
        for alignment in blast_record.alignments:  # Iterate over each alignment in the BLAST record
            for hsp in alignment.hsps:  # Iterate over each high-scoring pair (HSP) in the alignment
                query_cover = (hsp.align_length / blast_record.query_length) * 100  # Calculate query coverage
                # Extract organism from the title
                organism = alignment.title.split('[')[-1].strip(']') if '[' in alignment.title else "Unknown"
                
                # Append the results to the list
                results.append({
                    'title': alignment.title,
                    'accession': alignment.accession,
                    'organism': organism,
                    'query_cover': query_cover,
                    'e_value': hsp.expect, 
                    'identity': (hsp.identities / hsp.align_length) * 100, 
                    'score': hsp.score  
                })
                # Add accession to the list if not already present
                if alignment.accession not in accession_numbers:
                    accession_numbers.append(alignment.accession)
        
        # Sort results by protein query coverage
        results.sort(key=lambda x: x['query_cover'], reverse=True)  # Sort in descending order
        return results[:100], accession_numbers[:100]  # Return the top 100 results and accession numbers
        
    except Exception as e:
        # Print an error message if the BLAST search fails
        print(f"Error during BLAST search: {e}")
        return None  # Return None if there was an error

# Main execution
protein_id = input("What accession number do you want to search for? Eg. CEF77095.1: ")  # Prompts user fora protein accession number
sequence = get_protein_sequence(protein_id)  # Retrieves the protein sequence using the provided accession number

if sequence:  # Check if a sequence was retrieved
    print(f"Sequence length: {len(sequence)}")  # Print the length of the sequence
    print(f"Sequence:\n{sequence}")  # Print the sequence
    
    # Ask if the user wants to use this sequence for the BLAST search
    use_sequence = input("Do you want to use this sequence for the BLAST search? (yes/no): ").strip().lower()
    
    if use_sequence == 'yes':  # If the user wants to proceed with the BLAST search
        results, accessions = run_blast_search(sequence)  # Run the BLAST search with the retrieved sequence

        # Print results
        if results is not None and len(results) > 0:  # Check if results are not None and not empty
            print("\nTop 100 BLAST hits sorted by query coverage:")
            
            # Set fixed widths for specific columns
            accession_width = max(len("Accession"), max(len(hit['accession']) for hit in results)) + 2
            organism_width = max(len("Organism"), max(len(hit['organism']) for hit in results)) + 2
            query_cover_width = 12  # Fixed width based on title length
            identity_width = 12      # Fixed width based on title length
            e_value_width = max(len("E-value"), max(len(f"{hit['e_value']:.2e}") for hit in results)) + 2

            # Print header with dynamic widths
            header_format = f"{{:<{accession_width}}}{{:<{organism_width}}}{{:<{query_cover_width}}}{{:<{identity_width}}}{{:<{e_value_width}}}"
            print(header_format.format("Accession", "Organism", "Query Cover", "Identity", "E-value"))
            print("-" * (accession_width + organism_width + query_cover_width + identity_width + e_value_width))

            for hit in results:
                print(header_format.format(
                    hit['accession'],
                    hit['organism'],
                    f"{hit['query_cover']:.1f}%",
                    f"{hit['identity']:.1f}%",
                    f"{hit['e_value']:.2e}"
                ))
                
            # Condense the accession list into a new variable
            condensed_accession_list = [hit['accession'] for hit in results]

            # Print accession numbers in same order as BLAST results
            print("\nAccession numbers from BLAST results:")
            for i, hit in enumerate(results, 1):
                print(f"{i}. {hit['accession']}")
            
            print(f"\nTotal number of accessions: {len(condensed_accession_list)}")
        else:
            print("No results found or an error occurred during the BLAST search.")
    else:
        print("BLAST search was not performed.")
else:
    print("No sequence retrieved or an error occurred.")

What accession number do you want to search for? Eg. CEF77095.1:  CEF77095.1
Provide your email address (required by NCBI):  nw409@exeter.ac.uk


Sequence length: 514
Sequence:
MDPLVASELEKLGDGVPFRARPQHVHHTWARTFSSLPELFIQPESLVEVEKVVNLARKCRRRLVTTGCGHSPSNITCTSSWLINLDNFNKILSVNQETGVVAMEGGIRLYALCAELEKHGLTMPNLGSINEQSISGAISTGTHGSSLRHGLMSENIISLKVTLANGTTVFCSKDTKTDLFRAALLSLGAVGIITEVTFQAVPSFTLRWQQSVDTDYKMLESWNGDLWTQSEFVRVWWFPYTRRAVVWNAEKTDEELRDPPQSGYDGSIGYYVYHNLLYLAQHVPRILPWVEWLVFGMQYGFRNGTTSSAVQPSGKALLMNCLYSQFVNEWAIPLHKGPEALRRLSSWINHLTPEDPDYVPHNIPFSAEGLYVHAPVEVRVSDTTLTSNVRPYLDITADDGPTLYLNATLYRPYLMDPPCHERYYEGFEWLMKDLGGRPHWAKNFETSRVEIEAFYGKNLESFRAVRSDADPQGMFVGPWHRERIMEEGEGLELEEVEIRREKNNSGGLTTFGVI


Do you want to use this sequence for the BLAST search? (yes/no):  yes
Provide your email address (required by NCBI):  nw409@exeter.ac.uk


Running BLAST search (this may take a few minutes)...

Top 100 BLAST hits sorted by query coverage:
Accession     Organism                                    Query Cover Identity    E-value   
--------------------------------------------------------------------------------------------
XP_053005593  Fusarium falciforme                         100.2%      84.5%       0.00e+00  
XP_052916250  Fusarium keratoplasticum                    100.2%      83.7%       0.00e+00  
XP_046136113  Fusarium solani                             100.2%      84.1%       0.00e+00  
UPK90849      Fusarium solani-melongenae                  100.2%      83.7%       0.00e+00  
KAI8682984    Fusarium sp. Ph1                            100.2%      83.9%       0.00e+00  
KAJ4321086    Fusarium piperis                            100.2%      83.9%       0.00e+00  
KAI8681574    Fusarium keratoplasticum                    100.2%      83.7%       0.00e+00  
KAJ4205750    Fusarium falciforme                         100.2